In [4]:
import pandas as pd
import tensorflow as tf

# Note: Classification is an example of supervised Learning 

# Defining constants we'd be needing for the Datasets
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# Keras used here to input the Datasets in to panda dataframe
train_path = tf.keras.utils.get_file("iris_training.csv", './data/iris_training.csv')
test_path = tf.keras.utils.get_file("iris_test.csv", './data/iris_test.csv')

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')

# displaying the first few data
train.head()


2023-03-19 17:06:43.300185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [5]:
# Training the Model
def input_fn(features, labels, training=True, batch_size=256): 
    #convert the input into Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # shuffle the data when in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [6]:
# work on feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [7]:
# Building the Model  -used Deep Neural Network Classifier here instead of linear classification
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    # introducing hidden layers -- still yet to grasp this concept
    hidden_units = [30, 10],
    # having the models choose between three classes
    n_classes =3
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpu4e37v2y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


2023-03-19 17:06:46.907696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# train the model using the classfier
classifier.train(
    input_fn= lambda: input_fn(train, train_y, training=True),
    steps=5000
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2023-03-19 17:06:48.153819: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-19 17:06:48.165395: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT64
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'dnn/zero_fraction/cond/output/_18'


INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpu4e37v2y/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.627623, step = 0
INFO:tensorflow:global_step/sec: 176.426
INFO:tensorflow:loss = 1.3785193, step = 100 (0.570 sec)
INFO:tensorflow:global_step/sec: 134.6
INFO:tensorflow:loss = 1.1169624, step = 200 (0.754 sec)
INFO:tensorflow:global_step/sec: 270.286
INFO:tensorflow:loss = 1.0028787, step = 300 (0.369 sec)
INFO:tensorflow:global_step/sec: 186.831
INFO:tensorflow:loss = 0.90561485, step = 400 (0.521 sec)
INFO:tensorflow:global_step/sec: 582.208
INFO:tensorflow:loss = 0.8541757, step = 500 (0.174 sec)
INFO:tensorflow:global_step/sec: 255.285
INFO:tensorflow:loss = 0.80183053, step = 600 (0.390 sec)
INFO:tensorflow:global_step/sec: 249.363
INFO:tensorflow:loss = 0.7690017, step = 700 (0.405 sec)
INFO

In [9]:
# Testing the Accuracy of the alg.
eval_result = classifier.evaluate(input_fn= lambda: input_fn(test, test_y, training=False))
print('\n Test Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-19T17:07:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpu4e37v2y/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.99246s
INFO:tensorflow:Finished evaluation at 2023-03-19-17:07:07
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6333333, average_loss = 0.52131826, global_step = 5000, loss = 0.52131826
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpu4e37v2y/model.ckpt-5000

 Test Accuracy: 0.633



In [10]:
# A Predictive Model : Users inputs in the numerical feats of the Flower 
# and it predicts what class of flower it is

def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type Numeric Values as prompted: ")
for feature in features: 
    valid = True
    while valid:
        val  = input(feature + ": ")
        if not val.isdigit(): valid = False
        # placing the Numerical Inputs in here
        predict[feature] = [float(val)]
        
predictions = classifier.predict(input_fn=lambda: input_fn(predict))
# placing every prediction into dictionary
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability
    ))

Please type Numeric Values as prompted: 
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpu4e37v2y/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (98.8%)
